In [ ]:
# -*- coding: utf-8 -*-
"""
================================================================================
SCRIPT UNIFICADO DE TREINAMENTO DE MODELOS PREDITIVOS
================================================================================

Objetivo:
---------
Este script centraliza o processo de treinamento, avaliação e salvamento de
múltiplos modelos de Machine Learning para prever indicadores socioeconômicos
e de saúde com base em dados históricos.

Modelos Treinados:
------------------
1. PIB per Capita (R$)
2. VAB Agropecuária (R$ 1.000)
3. Total de Benefícios Básicos (Bolsa Família)
4. Internações por Doenças Respiratórias

Metodologia:
------------
1.  **Carregamento de Dados:** Carrega os datasets necessários.
2.  **Engenharia de Features:** Cria variáveis de lag (defasagem) e de taxa de
    crescimento para capturar tendências temporais. Esta é a etapa mais
    importante para transformar dados de série temporal em um formato que
    modelos de regressão como o LightGBM possam aprender.
3.  **Divisão Temporal:** Os dados são divididos em conjuntos de treino e teste
    com base no ano. Isso simula um cenário real onde prevemos o futuro
    (ex: ano de 2022) usando apenas informações do passado (até 2021).
    Uma divisão aleatória seria incorreta aqui, pois levaria a "vazamento de dados"
    do futuro para o treino.
4.  **Treinamento do Modelo:** Utiliza o LightGBM, um algoritmo de Gradient Boosting
    eficiente e de alta performance, ideal para dados tabulares.
5.  **Avaliação:** Mede o desempenho dos modelos usando RMSE (Erro Quadrático Médio
    da Raiz) e R² (Coeficiente de Determinação).
6.  **Análise de Importância (SHAP):** Gera gráficos SHAP para entender quais
    features são mais influentes nas previsões de cada modelo.
7.  **Salvamento:** Salva os modelos treinados e a lista de colunas utilizadas,
    prontos para serem carregados pelo dashboard Streamlit.

Autor: Seu Nome
Data: DD/MM/AAAA
"""

import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.metrics import mean_squared_error, r2_score
import shap
import joblib
import os
import matplotlib.pyplot as plt

# --- 1. CONFIGURAÇÃO E CONSTANTES ---
# Centralizar as configurações facilita a manutenção do código.

# Caminhos para os arquivos de dados
DATA_DIR = '../data/RESULTADOS/'
MODELS_DIR = '../models/'

# Garante que o diretório de modelos exista
os.makedirs(MODELS_DIR, exist_ok=True)

# Definição das features que serão usadas para criar lags e taxas de crescimento.
# Estas são as variáveis preditoras principais que capturam a dinâmica temporal.
FEATURES_PARA_LAG_BASE = [
    'Desmatamento (km²)', 'PIB per capita (R$)', 'VAB Agropecuária (R$ 1.000)',
    'Focos de Queimada', 'Área plantada soja (ha)', 'Total Rebanho (Bovino)'
]

# Features adicionais para os modelos específicos que também precisam de lag
FEATURES_PARA_LAG_BENEFICIOS = FEATURES_PARA_LAG_BASE + ['Total de Benefícios Básicos (Bolsa Família)']
FEATURES_PARA_LAG_RESPIRATORIO = FEATURES_PARA_LAG_BASE + ['Total de Benefícios Básicos (Bolsa Família)', 'Internações por Doenças Respiratórias']

# Parâmetros do LightGBM. Foram otimizados para um bom equilíbrio entre performance e tempo de treino.
LGB_PARAMS = {
    'objective': 'regression_l1',  # Otimiza para o Erro Absoluto Médio (MAE), mais robusto a outliers
    'metric': 'rmse',              # Métrica de avaliação durante o treino
    'n_estimators': 1500,          # Número máximo de árvores de decisão
    'learning_rate': 0.03,         # Taxa de aprendizado (passo do gradiente)
    'feature_fraction': 0.8,       # Porcentagem de features a ser usada em cada árvore
    'bagging_fraction': 0.8,       # Porcentagem de dados a ser usada em cada iteração (reduz overfitting)
    'bagging_freq': 1,
    'lambda_l1': 0.1,              # Regularização L1
    'lambda_l2': 0.1,              # Regularização L2
    'verbose': -1,                 # Suprime mensagens durante o treino
    'n_jobs': -1,                  # Usa todos os cores de CPU disponíveis
    'seed': 42                     # Garante a reprodutibilidade dos resultados
}

# --- 2. FUNÇÕES AUXILIARES ---

def carregar_dados(caminho_arquivo):
    """Carrega e prepara o dataframe inicial."""
    print(f"Carregando dados de '{caminho_arquivo}'...")
    try:
        df = pd.read_csv(caminho_arquivo)
        # Ordenar por município e ano é CRUCIAL para que os cálculos de lag/shift funcionem corretamente.
        df = df.sort_values(by=['Município', 'Ano']).reset_index(drop=True)
        print("Dados carregados com sucesso.")
        return df
    except FileNotFoundError:
        print(f"ERRO: Arquivo '{caminho_arquivo}' não encontrado. Abortando.")
        return None

def criar_features_temporais(df, features_para_lag):
    """
    Cria features de lag (ano anterior) e de crescimento (variação percentual).
    Esta função é o coração da engenharia de features para séries temporais.
    """
    print("Iniciando engenharia de features (lags e crescimento)...")
    df_featured = df.copy()
    for feature in features_para_lag:
        if feature in df_featured.columns:
            # Lag de 1 ano: Traz o valor do ano anterior para a linha atual.
            # O groupby('Município') garante que o lag não "vaze" de um município para outro.
            df_featured[f'{feature}_lag1'] = df_featured.groupby('Município')[feature].shift(1)
            
            # Taxa de crescimento: Calcula a variação percentual em relação ao ano anterior.
            # Isso captura a "aceleração" ou "desaceleração" da variável.
            df_featured[f'{feature}_growth'] = df_featured.groupby('Município')[feature].pct_change()
        else:
            print(f"Aviso: A coluna '{feature}' não foi encontrada para criar lag.")
            
    # Após calcular 'pct_change', podem surgir valores infinitos (divisão por zero).
    # Substituímos por 0 para evitar problemas no modelo.
    df_featured.replace([np.inf, -np.inf], 0, inplace=True)
    print("Engenharia de features concluída.")
    return df_featured

def treinar_avaliar_e_salvar_modelo(df_completo, nome_alvo, features_usadas, nome_modelo_amigavel, caminho_modelo, caminho_colunas):
    """
    Função genérica para executar o ciclo completo de treinamento de um modelo.
    """
    print("\n" + "="*80)
    print(f"INICIANDO TREINAMENTO PARA O MODELO: {nome_modelo_amigavel.upper()}")
    print("="*80)

    # --- 3.1. Preparação dos Dados Específicos do Modelo ---
    print(f"Definindo a variável alvo: '{nome_alvo}' do ano seguinte.")
    # O alvo (y) é o valor da variável no ano seguinte (t+1).
    # Usamos os dados do ano atual (t) para prever o ano seguinte.
    df_modelo = df_completo.copy()
    df_modelo['target'] = df_modelo.groupby('Município')[nome_alvo].shift(-1)

    # Removemos linhas com valores NaN. Elas são geradas pelo `shift` do lag e do alvo.
    # Essas linhas não podem ser usadas para treinar ou testar o modelo.
    df_modelo.dropna(subset=['target'] + [f for f in df_modelo.columns if '_lag1' in f or '_growth' in f], inplace=True)
    
    y = df_modelo['target']
    # 'Município' e o próprio alvo original são removidos das features (X).
    X = df_modelo.drop(columns=['target', 'Município', nome_alvo])
    # Mantemos o 'Ano' por enquanto para fazer a divisão temporal.
    
    print(f"Total de amostras válidas para o modelo: {len(X)}")

    # --- 3.2. Divisão Temporal (Treino/Teste) ---
    print("Realizando divisão temporal: treino até 2021, teste em 2022...")
    ano_corte = 2021
    
    X_train = X[X['Ano'] <= ano_corte]
    y_train = y[X['Ano'] <= ano_corte]
    X_test = X[X['Ano'] > ano_corte]
    y_test = y[X['Ano'] > ano_corte]

    # Agora que a divisão foi feita, o 'Ano' já cumpriu sua função e pode ser removido das features.
    X_train = X_train.drop(columns=['Ano'])
    X_test = X_test.drop(columns=['Ano'])
    
    # Garantir que as colunas de treino e teste estejam na mesma ordem
    model_columns = X_train.columns.tolist()
    X_test = X_test[model_columns]

    print(f"Tamanho do conjunto de treino: {len(X_train)} amostras.")
    print(f"Tamanho do conjunto de teste: {len(X_test)} amostras.")

    # --- 3.3. Treinamento do Modelo LightGBM ---
    print("Treinando o modelo LightGBM...")
    model = lgb.LGBMRegressor(**LGB_PARAMS)
    
    # O `early_stopping` monitora a performance no set de validação (X_test, y_test)
    # e para o treino se a performance não melhorar por 100 rodadas, evitando overfitting.
    model.fit(X_train, y_train,
              eval_set=[(X_test, y_test)],
              eval_metric='rmse',
              callbacks=[lgb.early_stopping(100, verbose=False)])

    # --- 3.4. Avaliação de Performance ---
    print("Avaliando o modelo no conjunto de teste...")
    predictions = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, predictions))
    r2 = r2_score(y_test, predictions)

    print("\n--- RESULTADOS DE PERFORMANCE ---")
    print(f"  - RMSE (Erro Quadrático Médio da Raiz): {rmse:,.2f}")
    print(f"  - R² (Coeficiente de Determinação):     {r2:.3f}")
    print("---------------------------------")
    
    # --- 3.5. Salvamento do Modelo e Colunas ---
    print("Salvando o modelo e a lista de colunas...")
    joblib.dump(model, caminho_modelo)
    joblib.dump(model_columns, caminho_colunas)
    print(f"  - Modelo salvo em: '{caminho_modelo}'")
    print(f"  - Colunas salvas em: '{caminho_colunas}'")

    # --- 3.6. Análise de Importância com SHAP ---
    print("Gerando análise de importância das features com SHAP...")
    explainer = shap.TreeExplainer(model)
    shap_values = explainer(X_test)

    # Gráfico de Barras: Importância média de cada feature
    plt.figure()
    shap.summary_plot(shap_values, X_test, plot_type="bar", show=False)
    plt.title(f'Importância Média das Features - {nome_modelo_amigavel}')
    plt.tight_layout()
    plt.show()

    # Gráfico Beeswarm: Impacto do valor de cada feature na previsão
    plt.figure()
    shap.summary_plot(shap_values, X_test, show=False)
    plt.title(f'Impacto Detalhado das Features - {nome_modelo_amigavel}')
    plt.tight_layout()
    plt.show()
    
    print(f"TREINAMENTO PARA '{nome_modelo_amigavel.upper()}' CONCLUÍDO.")

In [ ]:
# --- 4. BLOCO DE EXECUÇÃO PRINCIPAL ---
if __name__ == "__main__":
    
    # Carregar os dois datasets principais
    df_path = os.path.join(DATA_DIR, 'df_final2.csv')
    
    df = carregar_dados(df_path)

    if df is None or df is None:
        exit()

    df_featured = criar_features_temporais(df, FEATURES_PARA_LAG_RESPIRATORIO)

    # --- Treinar Modelo 1: PIB per Capita ---
    treinar_avaliar_e_salvar_modelo(
        df_completo=df_featured,
        nome_alvo='PIB per capita (R$)',
        features_usadas=FEATURES_PARA_LAG_BASE,
        nome_modelo_amigavel='PIB per Capita',
        caminho_modelo=os.path.join(MODELS_DIR, 'modelo_pib_pred.joblib'),
        caminho_colunas=os.path.join(MODELS_DIR, 'model_columns.joblib')
    )

    # --- Treinar Modelo 2: VAB Agropecuária ---
    treinar_avaliar_e_salvar_modelo(
        df_completo=df_featured,
        nome_alvo='VAB Agropecuária (R$ 1.000)',
        features_usadas=FEATURES_PARA_LAG_BASE,
        nome_modelo_amigavel='VAB Agropecuária',
        caminho_modelo=os.path.join(MODELS_DIR, 'modelo_vab_pred.joblib'),
        caminho_colunas=os.path.join(MODELS_DIR, 'vab_model_columns.joblib')
    )
    
    # --- Treinar Modelo 3: Benefícios Sociais ---
    treinar_avaliar_e_salvar_modelo(
        df_completo=df_featured,
        nome_alvo='Total de Benefícios Básicos (Bolsa Família)',
        features_usadas=FEATURES_PARA_LAG_BENEFICIOS,
        nome_modelo_amigavel='Benefícios Sociais',
        caminho_modelo=os.path.join(MODELS_DIR, 'modelo_beneficios_pred.joblib'),
        caminho_colunas=os.path.join(MODELS_DIR, 'beneficios_model_columns.joblib')
    )

    # --- Treinar Modelo 4: Saúde Respiratória ---
    treinar_avaliar_e_salvar_modelo(
        df_completo=df_featured,
        nome_alvo='Internações por Doenças Respiratórias',
        features_usadas=FEATURES_PARA_LAG_RESPIRATORIO,
        nome_modelo_amigavel='Saúde Respiratória',
        caminho_modelo=os.path.join(MODELS_DIR, 'modelo_respiratorio_pred.joblib'),
        caminho_colunas=os.path.join(MODELS_DIR, 'respiratorio_model_columns.joblib')
    )

    print("\n" + "#"*80)
    print("TODOS OS MODELOS FORAM TREINADOS E SALVOS COM SUCESSO!")
    print(f"Os arquivos estão prontos no diretório: '{MODELS_DIR}'")
    print("#"*80)

Carregando dados de 'data/RESULTADOS/df_final2.csv'...
ERRO: Arquivo 'data/RESULTADOS/df_final2.csv' não encontrado. Abortando.
Iniciando engenharia de features (lags e crescimento)...


AttributeError: 'NoneType' object has no attribute 'copy'

: 